In [2]:
!git clone https://github.com/Kesireddy-rohini/voting-system/

Cloning into 'voting-system'...
remote: Enumerating objects: 374, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 374 (delta 28), reused 58 (delta 12), pack-reused 275 (from 1)
Receiving objects: 100% (374/374), 2.06 MiB | 7.73 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [3]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00


In [4]:
from google.colab import files

# This will prompt you to upload the file
uploaded = files.upload()

Saving politics_qa.csv to politics_qa.csv


In [5]:
data = pd.read_csv('politics_qa.csv')

NameError: name 'pd' is not defined

In [6]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForQuestionAnswering, Trainer, TrainingArguments
import torch

# Load the dataset
data = pd.read_csv('/content/politics_qa.csv')

# Check for required columns
if 'question' not in data.columns or 'answer' not in data.columns:
    raise ValueError("The CSV must contain 'question' and 'answer' columns.")

# Create a general context for questions
context = "This document discusses various political issues and positions in the 2024 elections, including economic policies, healthcare, and reform initiatives."

# Prepare dataset for Hugging Face
def prepare_data(data):
    questions = data['question'].tolist()
    answers = data['answer'].tolist()

    # Use the same context for all questions
    contexts = [context] * len(questions)

    return {
        'question': questions,
        'context': contexts,
        'answers': [{'text': ans, 'start': context.find(ans)} for ans in answers]
    }

# Create dataset
dataset_dict = prepare_data(data)
dataset = Dataset.from_dict(dataset_dict)

# Split the dataset into training and validation sets
train_test_split = dataset.train_test_split(test_size=0.2)  # 80% train, 20% validation

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Tokenize the inputs
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['question'], examples['context'], truncation=True, padding='max_length', max_length=512
    )

    start_positions = []
    end_positions = []
    for i in range(len(examples['answers'])):
        start = examples['answers'][i]['start']
        end = start + len(examples['answers'][i]['text'])
        start_positions.append(start)
        end_positions.append(end)

    tokenized['start_positions'] = start_positions
    tokenized['end_positions'] = end_positions
    return tokenized

# Tokenize the train and validation sets
tokenized_train = train_test_split['train'].map(tokenize_function, batched=True)
tokenized_val = train_test_split['test'].map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',  # Use eval_strategy instead of evaluation_strategy
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,  # Provide the eval dataset here
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    # Handle case where answer may not be found
    if answer_start >= answer_end:
        return "No answer found"

    # Convert token IDs to string
    answer_tokens = inputs['input_ids'][0][answer_start:answer_end]
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(answer_tokens))
    return answer.strip()

# Example usage
if __name__ == "__main__":
    question = "What are the main issues in the 2024 election?"
    answer = answer_question(question, context)
    print("Answer:", answer)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Map:   0%|          | 0/12 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [7]:
import unittest
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Mock data for testing
mock_data = {
    'question': ['What are the main issues in the 2024 election?'],
    'answer': ['economic policies']
}
mock_df = pd.DataFrame(mock_data)

class TestQuestionAnsweringModel(unittest.TestCase):

    @classmethod
    def setUpClass(cls):
        # This method will run once for the entire test class
        cls.data = mock_df
        cls.context = "This document discusses various political issues and positions in the 2024 elections, including economic policies, healthcare, and reform initiatives."
        cls.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        cls.model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

    def test_load_data(self):
        # Check if the DataFrame contains the required columns
        self.assertIn('question', self.data.columns)
        self.assertIn('answer', self.data.columns)

    def test_prepare_data(self):
        # Prepare the data and check the output structure
        def prepare_data(data):
            questions = data['question'].tolist()
            answers = data['answer'].tolist()
            contexts = [self.context] * len(questions)

            return {
                'question': questions,
                'context': contexts,
                'answers': [{'text': ans, 'start': self.context.find(ans)} for ans in answers]
            }

        prepared_data = prepare_data(self.data)
        self.assertEqual(len(prepared_data['question']), 1)
        self.assertEqual(len(prepared_data['context']), 1)
        self.assertEqual(prepared_data['answers'][0]['text'], 'economic policies')

    def test_tokenize_function(self):
        def tokenize_function(examples):
            tokenized = self.tokenizer(
                examples['question'], examples['context'], truncation=True, padding='max_length', max_length=512
            )
            start_positions = [self.context.find(ans['text']) for ans in examples['answers']]
            end_positions = [start + len(ans['text']) for start, ans in zip(start_positions, examples['answers'])]
            tokenized['start_positions'] = start_positions
            tokenized['end_positions'] = end_positions
            return tokenized

        examples = {
            'question': ['What are the main issues in the 2024 election?'],
            'context': [self.context],
            'answers': [{'text': 'economic policies', 'start': self.context.find('economic policies')}]
        }
        tokenized_output = tokenize_function(examples)
        self.assertIn('start_positions', tokenized_output)
        self.assertIn('end_positions', tokenized_output)
        self.assertEqual(tokenized_output['start_positions'][0], self.context.find('economic policies'))

    def test_answer_question(self):
        def answer_question(question, context):
            inputs = self.tokenizer(question, context, return_tensors='pt', truncation=True)
            with torch.no_grad():
                outputs = self.model(**inputs)

            answer_start = torch.argmax(outputs.start_logits)
            answer_end = torch.argmax(outputs.end_logits) + 1

            if answer_start >= answer_end:
                return "No answer found"

            answer_tokens = inputs['input_ids'][0][answer_start:answer_end]
            answer = self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(answer_tokens))
            return answer.strip()

        question = "What are the main issues in the 2024 election?"
        answer = answer_question(question, self.context)
        self.assertIn('economic policies', answer)

if __name__ == '__main__':
    unittest.main()


E
ERROR: /root/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
